# Gather insights from the experiments run on GCP

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from common.utils import check_missing_experiments, join_df
from common.metrics import *

## Define the TTA Formula 

In [ ]:
# Calculate time to accuracy for different accuracies
def tta_crossbow(acc:int, df: pd.DataFrame, acc_column='accuracy', time_column='epoch_duration'):
    """Computes the tta as in the crossbow paper
    where the tta is the median of the last 5 epochs"""

    res = []
    for _, row in df.iterrows():
        done = False
        dur, accuracy = row[time_column], row[acc_column]
        
        for idx, (t, a) in enumerate(zip(dur, accuracy[:len(dur)])):
            
            # if there are less than 5 elements behind, continue
            if idx < 4:
                continue
                
            # calculate the median of the next five elements
            if np.median(accuracy[idx - 4:idx+1]) >= acc:
                res.append(t)
                done = True
                break

        if not done:
            res.append(np.nan)

    return res


def tta(acc:int, df:pd.DataFrame,  acc_column='accuracy', time_column='epoch_duration'):
    
    
    res = []
    for _, row in df.iterrows():
        done=False
        dur, accuracy = row[time_column], row[acc_column]
        
        for idx, (t, a) in enumerate(zip(dur, accuracy[:len(dur)])):
         
            if a >= acc:
                res.append(t)
                done = True
                break

        if not done:
            res.append(np.nan)

    return res
        

In [ ]:
# function to get the best results and the parameters for them
def get_best_combinations(ml: pd.DataFrame, column: str):
    """
    Given a dataframe, return the options with the best TTA and the column to optimize
    """
    
    # get only the non object columns
#     ml = ml[[col for col in ml.columns if ml[col].dtype != 'object']]

    # get the minimum rown of each of the batch sizes
    min_rows = ml.groupby('batch_size')[column].min()
    best = ml.loc[ml[column].isin(min_rows.values)][['k', 'batch_size', 'parallelism']]

    # get the rows with those combinations
    return ml.merge(best)

# KubeML Experiments

How to treat the kubeml experiments

### Load the train results

In [ ]:
import glob
glob.glob('./results/resn')

In [ ]:
df = join_df('./results/resnet32/128-no-momentum/')
df = df[df.default_parallelism > 1]

In [ ]:
df['momentum'] = False
# df=pd.concat([df,mom], ignore_index=True)

In [ ]:
df

In [ ]:
# get duplicated rows
df[df.duplicated(['hash'], keep=False)]

Define the extra variables

In [ ]:
# Set the acc to the final accuracy
df['acc'] = df.accuracy.map(lambda a: a[-1])

# Set the time to the sum of the epoch durations
df['time'] = df.epoch_duration.map(lambda t: t[-1])

# Set the parallelism to the first since it is constant
df.parallelism = df.parallelism.map(lambda l:l[0])

# change -1 to inf so the order is right in the plot
df.k = df.k.map(lambda val: float('inf') if val == -1 else val)

df['global_batch'] = df.batch_size * df.parallelism




In [ ]:
# compute the ttas
df['tta_80'] = tta(80, df)
df['tta_cross_80'] = tta_crossbow(80, df)

# compute the ttas
df['tta_67'] = tta(67, df)
df['tta_cross_67'] = tta_crossbow(67, df)

# compute the ttas
df['tta_70'] = tta(70, df)
df['tta_cross_70'] = tta_crossbow(70, df)

### For the metrics in resnet

In [ ]:
metrics = join_df('./results/resnet_sparse/metrics')
m = metrics.rename(columns={'exp_name':'id'})

### For the metrics in the lenet

The first replication does not have the proper format, so we need to reformat it and combine it with the train results

In [ ]:
metrics1 = join_df('./results/lenet/metrics/1/')
metrics2 = join_df('./results/lenet/metrics/2/')
metrics3 = join_df('./results/lenet/metrics/3/')

cpu = metrics1.groupby('exp_name')['cpu'].apply(list)
mem = metrics1.groupby('exp_name')['mem'].apply(list)
exps = metrics1.groupby('exp_name')['exp_name']

metrics1 = pd.DataFrame({
    'cpu':cpu,
    'mem':mem
})
metrics1['exp_name'] = metrics1.index

# concat all metrics and rename the exp_name as in the train
m = pd.concat([metrics1, metrics2, metrics3], ignore_index=True)
m.rename(columns={'exp_name':'id'}, inplace=True)


Now we need to add extra summary columns to the metrics

In [ ]:
# functions to compute the mean of each and add columns
m['mem'] = m['mem'].map(lambda l: l[0] if isinstance(l[0], list) else l)
m['cpu'] = m['cpu'].map(lambda l: l[0] if isinstance(l[0], list) else l)

# cpu util
m['cpu_mean'] = m['cpu'].map(lambda l: np.mean([s.percent for s in l]))

# gpu mean mem and util
m['gpu_0_mean_usage'] = m['gpu_0'].map(lambda l: np.mean([s.load for s in l if s.mem_used != 0]) if not isinstance(l, float) else l)
m['gpu_1_mean_usage'] = m['gpu_1'].map(lambda l: np.mean([s.load for s in l if s.mem_used !=0]) if not isinstance(l, float) else l)
m['gpu_0_mean_memory'] = m['gpu_0'].map(lambda l: np.mean([s.mem_used for s in l if s.mem_used != 0])if not isinstance(l, float) else l)
m['gpu_1_mean_memory'] = m['gpu_1'].map(lambda l: np.mean([s.mem_used for s in l if s.mem_used != 0])if not isinstance(l, float) else l)
m['gpu_usage'] = (m['gpu_0_mean_usage'] + m['gpu_1_mean_usage']) /2

# memory mean util
m['mem_mean'] = m['mem'].map(lambda l: np.mean([s.percent for s in l]))

#### Combine and Save the whole experiment

In [ ]:
d = df.merge(m, on='id')
d.to_pickle('./dataframes/resnet_kubeml_sparse.pkl')


# TensorFlow Experiments

How to treat the TF experiments

In [ ]:
folder = 'lenet'
df = join_df(f'./results/tf/{folder}/train_full/1/', f'./results/tf/{folder}/train_full/2', f'./results/tf/{folder}/train_full/3')

# Set the acc to the final accuracy
df['acc'] = df.val_accuracy.map(lambda a: a[-1])

# Set the time to the sum of the epoch durations
df['time'] = df.times.map(lambda t: t[-1])



In [ ]:
#tta_lenet
df['tta_99'] = tta(0.99, df, time_column='times', acc_column='val_accuracy')
df['tta_cross_99'] = tta_crossbow(0.99, df, time_column='times', acc_column='val_accuracy')

In [ ]:
# Define the TTA
df['tta_67'] = tta(0.67, df, time_column='times', acc_column='val_accuracy')
df['tta_cross_67'] = tta_crossbow(0.67, df, time_column='times', acc_column='val_accuracy')

df['tta_69'] = tta(0.69, df, time_column='times', acc_column='val_accuracy')
df['tta_cross_69'] = tta_crossbow(0.69, df, time_column='times', acc_column='val_accuracy')

df['tta_70'] = tta(0.70, df, time_column='times', acc_column='val_accuracy')
df['tta_cross_70'] = tta_crossbow(0.70, df, time_column='times', acc_column='val_accuracy')


## Merge the metrics from different  folders


In [ ]:
m = join_df(f'./results/tf/{folder}/metrics_full/1/', f'./results/tf/{folder}/metrics_full/2', f'./results/tf/{folder}/metrics_full/3')

In [ ]:
# functions to compute the mean of each and add columns
m['mem'] = m['mem'].map(lambda l: l[0] if isinstance(l[0], list) else l)
m['cpu'] = m['cpu'].map(lambda l: l[0] if isinstance(l[0], list) else l)

# cpu util
m['cpu_mean'] = m['cpu'].map(lambda l: np.mean([s.percent for s in l]))

# gpu mean mem and util
m['gpu_0_mean_usage'] = m['gpu_0'].map(lambda l: np.mean([s.load for s in l if s.mem_used != 0]) if not isinstance(l, float) else l)
m['gpu_1_mean_usage'] = m['gpu_1'].map(lambda l: np.mean([s.load for s in l if s.mem_used !=0]) if not isinstance(l, float) else l)
m['gpu_0_mean_memory'] = m['gpu_0'].map(lambda l: np.mean([s.mem_used for s in l if s.mem_used != 0])if not isinstance(l, float) else l)
m['gpu_1_mean_memory'] = m['gpu_1'].map(lambda l: np.mean([s.mem_used for s in l if s.mem_used != 0])if not isinstance(l, float) else l)
m['gpu_usage'] = (m['gpu_0_mean_usage'] + m['gpu_1_mean_usage']) /2

# memory mean util
m['mem_mean'] = m['mem'].map(lambda l: np.mean([s.percent for s in l]))

Join on the index

In [ ]:
df = df.join(m)

Save the experiments

In [ ]:
df.to_pickle('./dataframes/lenet_full_tensorflow.pkl')

In [ ]:
df

# Visualization of Results


Set the color palette

In [ ]:
green_red_palette = ['#264653', '#2A9D8F', '#E9C46A', '#F4A261', '#E76F51']
blue_yellow_palette=  ['#0077b6', '#d62828', '#f77f00', '#fcbf49', '#eae2b7']
cool_p = ['#f87575', '#ffa9a3', '#b9e6ff', '#5c95ff', '#7e6c6c']
wall_p = ['#e63946', '#f1faee', '#a8dadc', '#457b9d', '#1d3557']

sns.palplot(sns.color_palette('gnuplot2'))


sns.set_theme(style='whitegrid', palette=blue_yellow_palette, )
sns.set_context('talk')
# sns.set_palette(blue_yellow_palette)

In [ ]:
a = sns.color_palette('gnuplot2').as_hex()
a

# Read the experiments file

Some of the dataframes by name

- resnet_new: is with extra epochs and proper preprocessing
- resnet_sparse: only holds the tests with sparse averaging
- folders with metrics/train_full hold the extra experiment to migrate to local batch comparison

### Load all of the dataframes

In [ ]:
# load the tf experiments
resnet = pd.read_pickle('./dataframes/resnet_new_full_tensorflow.pkl')
lenet = pd.read_pickle('./dataframes/lenet_full_tensorflow.pkl')

lenet.rename(columns={
    'loss':'train_loss',
    'val_accuracy':'accuracy',
    'val_loss':'validation_loss',
    'times':'epoch_duration',
    'accuracy':'train_accuracy',
    'val_accuracy':'accuracy'
}, inplace=True)
lenet['system'] = 'tensorflow'
lenet['acc'] = 100*lenet['acc']
lenet['accuracy'] = lenet['accuracy'].map(lambda l: [100*n for n in l])

# rename columns in the TF dataframes to adhere to the kubeml ones
resnet.rename(columns={
    'loss':'train_loss',
    'val_accuracy':'accuracy',
    'val_loss':'validation_loss',
    'times':'epoch_duration',
    'accuracy':'train_accuracy',
    'val_accuracy':'accuracy'
}, inplace=True)
resnet['system'] = 'tensorflow'
resnet['acc'] = 100*resnet['acc']
resnet['accuracy'] = resnet['accuracy'].map(lambda l: [100*n for n in l])


# set the columns of the 

# load the kubeml experiments
kuberesnet = pd.read_pickle('./dataframes/resnet_kubeml.pkl')
kuberesnet['model'] = 'resnet'
kuberesnet['system'] = 'kubeml'
# kuberesnet['tta_70'] = tta(70, kuberesnet)
# kuberesnet['tta_cross_70'] = tta_crossbow(70, kuberesnet)

kubelenet = pd.read_pickle('./dataframes/lenet_kubeml.pkl')
kubelenet['model'] = 'lenet'
kubelenet['system'] = 'kubeml'


In [ ]:
kubelenet

In [ ]:
# save the full resnet stuff
r.to_pickle('./dataframes/resnet_new_full.pkl')

In [ ]:
r = pd.concat([resnet, kuberesnet], ignore_index=True)
r
# resnet

### Create new columns for representation

- Final accuracy
- Total time taken
- Parallelism

### Plot the Correlations between the K, Batch and Parallelism with time and accuracy

In [ ]:
corr = df[['k', 'batch_size', 'parallelism', 'acc', 'time']].corr()
sns.heatmap(corr,
            annot=True,
)

# plt.savefig('./figures/resnet34/heat.png', dpi=300)


In [ ]:
df.sort_values('time')

mean = df.groupby('hash').mean()

### Get the max accuracies and times and check the parameters used

In [ ]:
# get the max accuracies
df[['k', 'parallelism', 'acc','batch_size','time']].sort_values(by='time', ascending=True)

## Calculate TTA with different accuracies



In [ ]:
s = df.sort_values('tta_cross_99')[['k', 'batch_size', 'parallelism', 'tta_cross_99', 'tta_99', 'acc', 'accuracy', 'epoch_duration']]

# plot the best
best = s.iloc[0]
best


x = range(1, len(best.accuracy)+1)
plt.figure()
plt.title(f'Best tta_99 (B={best.batch_size}, k={best.k}, P={best.parallelism})')
sns.lineplot(x=best.epoch_duration, y = best.accuracy)
sns.lineplot(x=best.epoch_duration, y= 99)
plt.scatter(best.tta_cross_99, 99, marker='X', s=60, c='r')
plt.xlabel('Time (s)', fontsize=15)
plt.ylabel('Accuracy (%)', fontsize=15)

# plt.savefig('./figures/gpu/best.png', dpi=300)


# Plot the resnet results KubeML vs Tensorflow

In [ ]:
#Define the colors used
tf_color = '#FF6F00'
kubecolor = '#316CE6'



## LeNet Results

In [ ]:
l = pd.read_pickle('./dataframes/lenet.pkl')

In [ ]:
len(l[l.system=='tensorflow'].iloc[0].accuracy)

## Get the best result from Lenet

In [ ]:
tf = l[l.system=='tensorflow']

tf['train_accuracy'] = tf.train_accuracy.map(lambda l: [100*n for n in l])


best = tf.sort_values('tta_99').head(1).iloc[0]




max_acc = np.max(best.accuracy)
max_acc




# before it was 10,5
plt.figure(figsize=(10, 7))

sns.lineplot(x=best.epoch_duration,
            y=99,
            color='red',
            )

# 1) plot the validation accuracy of TF
sns.lineplot(x=best.epoch_duration,
             y=best.accuracy,
             data=best,  
            label='Test Accuracy',
#              color=tf_color
            )

line = sns.lineplot(x=best.epoch_duration,
             y=best.train_accuracy,
             data=best,  
            label='Train Accuracy',
            color='gray',
            alpha=.6
            )


# plt.ylim([30, 80])
plt.xlabel('Time (sec)')
plt.ylabel("Accuracy (%)")

line.lines[2].set_linestyle((0, (15, 6)))

# sns.despine()

plt.tight_layout()
plt.savefig('./figures/paper/lenet-baseline.pdf')

### Plot the Accuracy and Train Loss Progression

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=False, figsize=(20, 6))

# get the best results
tf = l[l.system=='tensorflow']
ml = l[l.system =='kubeml']
sample = tf.iloc[5]
kubesample = ml.iloc[87]

# sort for tta69 and show

# 1) plot the validation accuracy of TF
sns.lineplot(x=sample.epoch_duration[:30],
             y=sample.accuracy[:30],
             data=sample, 
            ax=ax1, 
            label='TensorFlow',
            color='#FF6F00')

# 2) plot the validation accuracy of KubeML
sns.lineplot(x=kubesample.epoch_duration,
             y='accuracy',
             data=kubesample, 
            ax=ax1, 
            label='KubeML',
            color='#316CE6')

ax1.set_ylim([97, 100])
ax1.set_xlabel('Time (sec)')



# plot the line at 69%
sns.lineplot(x=kubesample.epoch_duration,
            y=99,
            ax=ax1,
            color='red')

# 3) Plot the train loss
sns.lineplot(x=sample.epoch_duration[:30],
             y=sample.train_loss[:30],
             data=sample, 
            ax=ax2, 
            label='TensorFlow',
            color='#FF6F00')

# 3) Plot the train loss
sns.lineplot(x=kubesample.epoch_duration,
             y='train_loss',
             data=kubesample, 
            ax=ax2, 
            label='KubeML',
            color='#316CE6')

ax2.set_xlabel('Time (sec)')



# 4) Plot the val loss
sns.lineplot(x=sample.epoch_duration[:30],
             y=sample.validation_loss[:30],
             data=sample, 
            ax=ax3, 
            label='TensorFlow',
            color='#FF6F00')

# 4) Plot the val loss
sns.lineplot(x=kubesample.epoch_duration,
             y='validation_loss',
             data=kubesample, 
            ax=ax3, 
            label='KubeML',
            color='#316CE6')

ax3.set_ylim([0, 0.3])

# plt.savefig('./figures/ppt/lenet_line.png', dpi=300)


### Plot the whole comparison

In [ ]:
# get only the results with k=-1 and 
# _l = l[((l.parallelism==4) | (l.parallelism.isna())) & ((l.k==float('inf')) | (l.k.isna())) ]

tf = l[l.system=='tensorflow']
ml = l[l.system=='kubeml']

tf['local_batch'] = (tf.batch_size/2).astype('int')

# get the best results
ml = get_best_combinations(ml, 'tta_99')
display(ml)
ml['local_batch'] = ml.batch_size
# print(ml.groupby('batch_size').median())

# combine them again
_l = pd.concat([tf, ml])
_l.system = _l.system.map(lambda val: 'KubeML' if val=='kubeml' else 'TensorFlow')
_l = _l[_l.local_batch >8]

# set the palette
sns.set_palette([tf_color, kubecolor], desat=0.7)
# sns.set_palette('gnuplot2')


# COMPARE THE LOSSES, THE VAL LOSS OF KUBEML SHOULD BE HIGHER (DO NOT KNOW THE TRAIN LOSS, BECAUSE OF EXPLORATION,
# WHILE STILL GIVING BEST ACCURACY (BETTER GENERALIZATION))
f, ax = plt.subplots(figsize=(10, 7))

# 1) Plot the comparison of tta between both
sns.barplot(x='local_batch', 
            y='tta_99', 
            data=_l, 
            hue='system', 
            errwidth=2.5,
#             ax=ax[0][0],
           capsize=.05)
plt.xlabel('Batch Size')
plt.ylabel('TTA (99%) (sec)')
sns.despine(left=True)
plt.ylim([0, 150])
# plt.setp(ax.get_legend().get_texts(), fontsize='15') 
# plt.setp(ax.get_legend().get_title(), fontsize='15')  
plt.tight_layout()
plt.savefig('./figures/paper/lenet/tta99.pdf')




# 3) Plot the difference train_loss
f, ax = plt.subplots(figsize=(10, 7))
_l['t_loss'] = _l['train_loss'].map(lambda l:l[-1])
sns.barplot(x='local_batch', 
            y='t_loss', 
            data=_l, 
            hue='system',
            errwidth=2.5,
#             ax=ax[1][0],
            capsize=.05,
               )
plt.xlabel('Batch Size')
plt.ylabel('Train Loss')
# plt.ylim([0, 0.8])
sns.despine(left=True)
plt.tight_layout()
plt.savefig('./figures/paper/lenet/train-loss.pdf')


# 4) Plot the val loss of both compared
# calculate the loss
f, ax = plt.subplots(figsize=(10, 7))
_l['loss'] = _l['validation_loss'].map(lambda l:l[-1])
sns.barplot(x='local_batch', 
            y='loss', 
            data=_l ,
            hue='system',
            errwidth=2.5,
#             ax=ax[1][1],
            capsize=.05)

plt.xlabel('Batch Size')
plt.ylabel('Validation Loss')
plt.ylim([0, 0.08])
sns.despine(left=True)
plt.tight_layout()
plt.savefig('./figures/paper/lenet/val-loss.pdf')


## Plot the difference in cpu and gpu utilization
# CPU util
f, ax = plt.subplots(figsize=(10, 7))
sns.barplot(x='local_batch', 
            y='cpu_mean', 
            data=_l ,
            capsize=.05,
            errwidth=2.5,
            hue='system',
#             ax=ax[2][0]
           )

plt.xlabel('Batch Size')
plt.ylabel('CPU Utilization (%)')
sns.despine(left=True)
plt.ylim([0, 15])
plt.tight_layout()
plt.savefig('./figures/paper/lenet/cpu.pdf')

# Global Batch Size relationship with tta
# _ml = r[r.system=='kubeml']
f, ax = plt.subplots(figsize=(10, 7))
sns.barplot(x='local_batch', 
            y='gpu_usage', 
            data=_l ,
            capsize=.05,
            errwidth=2.5,
            hue='system',
#             ax=ax[2][1]
           )

plt.xlabel('Batch Size')
plt.ylabel('GPU Usage')
plt.ylim([0, 1.15])
sns.despine(left=True)
plt.tight_layout()
plt.savefig('./figures/paper/lenet/gpu.pdf')
# plt.savefig('./figures/ppt/lenet-performance.png', dpi=400)

### Comparison of accuracy and time between K

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(20, 10))

sns.set_palette('gnuplot2', desat=0.8)



_kubelenet = l[(l.system == 'kubeml')]
# 1) plot the accuracy reached with different k
sns.barplot(x='batch_size',
           y='time',
           data=_kubelenet, 
           hue='parallelism',
           ax=ax[0],
           errcolor='gray',
           capsize=.05,
           errwidth=2.5)
ax[0].set_xlabel('Batch Size')
ax[0].set_ylabel('Time (sec)')


_kubelenet = l[(l.system == 'kubeml') & (l.parallelism==4)]
# 2) plot the time reached with different k
sns.barplot(x='batch_size',
           y='acc',
           data=_kubelenet, 
           hue='parallelism',
           ax=ax[1],
           errcolor='gray',
           capsize=.05,
           errwidth=2.5)
ax[1].set_xlabel('Batch Size')
ax[1].set_ylabel('Time (sec)')

plt.suptitle('Evolution of Time with Parallelism and K')

plt.savefig('./figures/ppt/lenet-time.png', dpi=400)

## Resnet Results

#### Plot the Train Loss and Val Accuracy of Tensorflow and KubeML


In [ ]:
#Define the colors used
tf_color = '#FF6F00'
kubecolor = '#316CE6'



In [ ]:
# set the palette
sns.set_palette([tf_color, kubecolor], desat=0.7)

In [ ]:
r = pd.read_pickle('./dataframes/resnet_sparse.pkl')

## Get the best result plot from Resnet

In [ ]:
tf = r[r.system=='tensorflow']

tf['train_accuracy'] = tf.train_accuracy.map(lambda l: [100*n for n in l])


best = tf.sort_values('tta_70').head(1).iloc[0]




max_acc = np.max(best.accuracy)
max_acc


plt.figure(figsize=(10, 7))
# 1) plot the validation accuracy of TF
sns.lineplot(x=best.epoch_duration,
             y=best.accuracy,
             data=best,  
            label='Test Accuracy',
#              color=tf_color
            )

l = sns.lineplot(x=best.epoch_duration,
             y=best.train_accuracy,
             data=best,  
            label='Train Accuracy',
            color='gray',
            alpha=.6
            )


sns.lineplot(x=best.epoch_duration,
            y=70,
            color='red',
            )
# plt.ylim([30, 80])
plt.xlabel('Time (sec)')
plt.ylabel("Accuracy (%)")

l.lines[1].set_linestyle((0, (15, 6)))

# sns.despine()

plt.tight_layout()
plt.savefig('./figures/paper/resnet-baseline.pdf')

In [ ]:
r[(r.batch_size==32) & (r.system=='kubeml') & (r.k == float('inf')) & (r.parallelism ==4)].iloc[0].epoch_duration

In [ ]:
print(sns.color_palette('gnuplot2').as_hex())

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=False, figsize=(20, 6))

# get the best results
tf = r[r.system=='tensorflow']
ml = r[r.system =='kubeml']
sample = r.iloc[tf.sort_values('tta_70').head(1).index[0]]
kubesample = r.iloc[ml.sort_values('tta_70').head(3).index[0]]

# kubesample =ml.iloc[2]
# sample = tf.iloc[3]

# sort for tta69 and show

# 1) plot the validation accuracy of TF
sns.lineplot(x=sample.epoch_duration[:50],
             y=sample.accuracy[:50],
             data=sample, 
            ax=ax1, 
            label='TensorFlow')

# 2) plot the validation accuracy of KubeML
sns.lineplot(x=kubesample.epoch_duration,
             y='accuracy',
             data=kubesample, 
            ax=ax1, 
            label='KubeML')

ax1.set_ylim([40, 80])



# plot the line at 70%
sns.lineplot(x=range(int(kubesample.time)),
            y=70,
            ax=ax1,
            color='red')


# 3) Plot the train loss
sns.lineplot(x=sample.epoch_duration[:30],
             y=sample.train_loss[:30],
             data=sample, 
            ax=ax2, 
            label='TensorFlow')

# 3) Plot the train loss
sns.lineplot(x=kubesample.epoch_duration,
             y='train_loss',
             data=kubesample, 
            ax=ax2, 
            label='KubeML')


# 4) Plot the val loss
sns.lineplot(x=sample.epoch_duration[:30],
             y=sample.validation_loss[:30],
             data=sample, 
            ax=ax3, 
            label='TensorFlow')

# 4) Plot the val loss
sns.lineplot(x=kubesample.epoch_duration,
             y='validation_loss',
             data=kubesample, 
            ax=ax3, 
            label='KubeML')

# plt.savefig('./figures/ppt/resnet_line.png', dpi=300)


### Plot the GPU, CPU and MEM usage based on k, batch and parallelism

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=False, figsize=(20, 6))
sns.set_palette('deep')

# 1) Plot the cpu usage
sns.barplot(x='batch_size', 
            y='cpu_mean', 
            data=ml, 
            hue='k', 
            errwidth=2.5,
            ax=ax1,
            capsize=.05)
# ax1.set_ylim([40, 100])
ax1.set_xlabel('Batch Size')
ax1.set_ylabel('CPU Usage (%)')


# 2) Plot the memory usage
sns.barplot(x='batch_size', 
            y='mem_mean', 
            data=ml, 
            hue='parallelism',
            errwidth=2.5,
            ax=ax2,
            capsize=.05,
               )
ax2.set_xlabel('Batch Size')
ax2.set_ylabel('Memory Usage (%)')


# 3) Plot the gpu usage
sns.barplot(x='batch_size', 
            y='gpu_usage', 
            data=ml ,
            hue='k',
            errwidth=2.5,
            ax=ax3,
            capsize=.05)

ax3.set_xlabel('Batch Size')
ax3.set_ylabel('GPU Usage')

In [ ]:
ml[(ml.parallelism==4) & (ml.k==float('inf')) & (ml.batch_size==128)].time

In [ ]:
_ml.groupby('batch_size')['tta_70'].mean()

### Plot the metrics of both 

In [ ]:
_s['parallelism'].value_counts()

In [ ]:
_ml[['batch_size', 'k', 'parallelism', 'epochs']]

In [ ]:
d = pd.read_pickle('./dataframes/resnet_new_full.pkl')

In [ ]:
d

In [ ]:
# get only the results with k=-1 and 
# _r = r[((r.parallelism==4) | (r.parallelism.isna())) & ((r.k==float('inf')) | (r.k.isna())) ]
# _r['t30'] = _r['time']

sns.set_palette([tf_color, kubecolor], desat=0.7)

sns.set_style('whitegrid')
plt.rc('font', size=22)

_r = pd.read_pickle('./dataframes/resnet_new_full.pkl')
_sp = pd.read_pickle('./dataframes/resnet_sparse.pkl')
_sp = _sp[_sp.system=='kubeml']

_s = pd.concat([_r,_sp])
_tf = _s[_s.system=='tensorflow']
_tf['local_batch'] = (_tf.batch_size/2).astype('int')

_ml = _s[_s.system=='kubeml']
_ml['local_batch'] = _ml['batch_size']
_ml = get_best_combinations(_ml, 'tta_70')


_r = pd.concat([_tf,_ml])
_r.system = _r.system.map(lambda val: 'KubeML' if val=='kubeml' else 'TensorFlow')
_r = _r[_r['local_batch'] >16]

# display(_ml)




# COMPARE THE LOSSES, THE VAL LOSS OF KUBEML SHOULD BE HIGHER (DO NOT KNOW THE TRAIN LOSS, BECAUSE OF EXPLORATION,
# WHILE STILL GIVING BEST ACCURACY (BETTER GENERALIZATION))
f, ax = plt.subplots(figsize=(10, 7))

# 1) Plot the comparison of tta between both
sns.barplot(x='local_batch', 
            y='tta_70', 
            data=_r, 
            hue='system', 
            errwidth=2.5,
#             ax=ax[0][0],
           capsize=.05)
plt.xlabel('Batch Size')
plt.ylabel('TTA (70%) (sec)')
sns.despine(left=True)
# plt.setp(ax.get_legend().get_texts(), fontsize='15') 
# plt.setp(ax.get_legend().get_title(), fontsize='15')  
plt.tight_layout()
plt.savefig('./figures/paper/resnet34/tta70.pdf')

# # create a function
# for (i, row), p in zip(parals.iterrows(), [p for p in ax[0][0].patches[1::2]]):
#     print(row, p)
#     ax[0][0].annotate(
#         str(int(row.parallelism)),
#         xy = ( p.get_x() + p.get_width(),  p.get_height() + 10))


# 3) Plot the difference train_loss
f, ax = plt.subplots(figsize=(10, 7))
_r['t_loss'] = _r['train_loss'].map(lambda l:l[-1])
sns.barplot(x='local_batch', 
            y='t_loss', 
            data=_r, 
            hue='system',
            errwidth=2.5,
#             ax=ax[1][0],
            capsize=.05,
               )
plt.xlabel('Batch Size')
plt.ylabel('Train Loss')
plt.ylim([0, 0.65])
sns.despine(left=True)
plt.tight_layout()
plt.savefig('./figures/paper/resnet34/train-loss.pdf')


# 4) Plot the val loss of both compared
# calculate the loss
f, ax = plt.subplots(figsize=(10, 7))
_r['loss'] = _r['validation_loss'].map(lambda l:l[-1])
sns.barplot(x='local_batch', 
            y='loss', 
            data=_r ,
            hue='system',
            errwidth=2.5,
#             ax=ax[1][1],
            capsize=.05)

plt.xlabel('Batch Size')
plt.ylabel('Validation Loss')
plt.ylim([0, 3.25])
sns.despine(left=True)
plt.tight_layout()
plt.savefig('./figures/paper/resnet34/val-loss.pdf')


## Plot the difference in cpu and gpu utilization
# CPU util
f, ax = plt.subplots(figsize=(10, 7))
sns.barplot(x='local_batch', 
            y='cpu_mean', 
            data=_r ,
            capsize=.05,
            errwidth=2.5,
            hue='system',
#             ax=ax[2][0]
           )

plt.xlabel('Batch Size')
plt.ylabel('CPU Utilization (%)')
sns.despine(left=True)
plt.ylim([0, 15])
plt.tight_layout()
plt.savefig('./figures/paper/resnet34/cpu.pdf')

# Global Batch Size relationship with tta
# _ml = r[r.system=='kubeml']
f, ax = plt.subplots(figsize=(10, 7))
sns.barplot(x='local_batch', 
            y='gpu_usage', 
            data=_r ,
            capsize=.05,
            errwidth=2.5,
            hue='system',
#             ax=ax[2][1]
           )

plt.xlabel('Batch Size')
plt.ylabel('GPU Usage')
plt.ylim([0, 1.15])
sns.despine(left=True)
plt.tight_layout()
plt.savefig('./figures/paper/resnet34/gpu.pdf')


In [ ]:
 ax.annotate(str(int(p)), 
                xy=(patch.get_x()+patch.get_width()/1.3, patch.get_height()+ 10), 
                ha='center', 
                fontsize=20, 
                color='red')

In [ ]:
def annotate_rectangles(df:pd.DataFrame):
    """Do the stuff"""

In [ ]:
parals = ml.groupby('batch_size').mean()[['parallelism', 'tta_70']]
# parals
for (i, row), p in zip(parals.iterrows(), [p for p in ax[0][0].patches[1::2]]):
    print(row, p)
    ax[0][0].annotate(
        str(int(row.parallelism)),
        xy = ( p.get_x() + p.get_width(),  p.get_height() + 10))

### Get the best results of KubeML by  Batch

In [ ]:
ml[ml.batch_size.eq(256) & ~ml.tta_70.isna()]['tta_70']

In [ ]:
ml = r[r.system=='kubeml']
ml = get_best_combinations(ml)
ml
_r = pd.concat([tf, ml], ignore_index=True)

In [ ]:
sns.set_palette('deep', 10)
sns.set_style('whitegrid')

def get_best_combinations(ml: pd.DataFrame):
    """
    Given a dataframe, return the options with the best TTA
    """
    
    # get only the non object columns
#     ml = ml[[col for col in ml.columns if ml[col].dtype != 'object']]

    # get the minimum rown of each of the batch sizes
    min_rows = ml.groupby('batch_size')['tta_70'].min()
    best = ml.loc[ml.tta_70.isin(min_rows.values)][['k', 'batch_size', 'parallelism']]

    # get the rows with those combinations
    return ml.merge(best)
    
f, ax = plt.subplots(figsize=(10, 6))

ml = r[r.system=='kubeml']
ml = get_best_combinations(ml)

sns.barplot(x='batch_size',
           y='tta_70', 
            data=ml,
           errwidth=2.5,
            capsize=.05, 
           )

ps = ml.groupby('batch_size').mean()[['parallelism', 'tta_70']]
for i, ((_, p, tta), patch) in enumerate(zip(ps.itertuples(), ax.patches)):
    print(patch)
    
    ax.annotate(str(int(p)), 
                xy=(patch.get_x()+patch.get_width()/1.3, patch.get_height()+ 10), 
                ha='center', 
                fontsize=20, 
                color='red')
    



### Comparison of Time with Parallelism and K 

(include these in paper!!)

In [ ]:
# f, (ax1, ax2) = plt.subplots(2, 2, figsize=(20, 15))

# sns.set_palette('muted',10)
sns.set_palette('gnuplot2', desat=0.8)



_kuberesnet = _s[(_s.system == 'kubeml')]
_kuberesnet['parallelism'] = _kuberesnet.parallelism.astype('int')
_kuberesnet['global_batch'] = _kuberesnet.global_batch.astype('int')
_kuberesnet['k'] = _kuberesnet.k.map(lambda v: int(v) if v != float('inf') else v)


f, ax = plt.subplots(figsize=(10,7))
print(_kuberesnet.parallelism.value_counts())
# 1) plot the accuracy reached with different k
sns.barplot(x='batch_size',
           y='time',
           data=_kuberesnet, 
            errwidth=2.5,
            capsize=.05,
            errcolor='gray',
           hue='parallelism',
           )
plt.xlabel('Batch Size')
plt.ylabel('Time (sec)')
sns.despine(left=True)

plt.tight_layout()
plt.savefig('./figures/paper/resnet34/batch-vs-time-by-parallelism.pdf')





# _kuberesnet = r[(r.system == 'kubeml') & (r.parallelism==4)]
# 2) plot the time reached with different k
f, ax = plt.subplots(figsize=(10,7))
sns.barplot(x='batch_size',
           y='time',
           data=_kuberesnet, 
            capsize=.05,
           hue='k',
            errcolor='gray',
        errwidth=2.5)
plt.xlabel('Batch Size')
plt.ylabel('Time (sec)')
sns.despine(left=True)


plt.tight_layout()
plt.savefig('./figures/paper/resnet34/batch-vs-time-by-k.pdf')



# _kuberesnet = r[(r.system == 'kubeml')]
# 1) plot the accuracy reached with different k
# sns.barplot(x='batch_size',
#            y='acc',
#            data=_kuberesnet, 
#            hue='parallelism',
#             errcolor='gray',
#             capsize=.05,
#             errwidth=2.5,
#            ax=ax2[0])
# ax2[0].set_xlabel('Batch Size')
# ax2[0].set_ylabel('Accuracy (%)')
# ax2[0].set_ylim([55, 73])


# _kuberesnet = r[(r.system == 'kubeml')]
# 1) plot the accuracy reached with different k

f, ax = plt.subplots(figsize=(10,7))
sns.barplot(x='global_batch',
           y='acc',
            errwidth=2.5,
            capsize=.05,
            errcolor='gray',
           data=_kuberesnet, 
           hue='parallelism')
plt.xlabel('Global Batch Size')
plt.ylabel('Accuracy (%)')
plt.ylim([55, 80])
sns.despine(left=True)

plt.tight_layout()
plt.savefig('./figures/paper/resnet34/global-batch-vs-acc.pdf')

# plt.suptitle('Evolution of Time with Parallelism and K (Resnet34)')

# plt.savefig('./figures/ppt/resnet-time.png', dpi=400)

In [ ]:
g = _r.loc[(_r.batch_size==32) & (_r.system=='kubeml')]
for a, t in zip(g.iloc[0].accuracy, g.iloc[0].epoch_duration):
    print(t, a)

---

# Analize results using ResNet32 for 200 epochs

In [ ]:
# load the dataframes
r32 = pd.read_pickle('./dataframes/resnet32-200.pkl')
r128 = pd.read_pickle('./dataframes/resnet32-200-128.pkl')

### Analysis of the batch 32 

In [ ]:
sns.set_palette('gnuplot2')

# get an intuition of the comparison with and without momentum by plotting the accuracy progression
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset

f, ax = plt.subplots(figsize=(14,10))

# plot all the lines
for _, row in r32.sort_values('k').iterrows():
    if not row.momentum:
        sns.lineplot(
            x=row.epoch_duration,
            y=row.accuracy,
            label=f'{int(row.k) if row.k != float("inf") else row.k}'
        )
        
# set the axes names
ax.set_xlabel('Time (sec)')
ax.set_ylabel('Accuracy (%)')
ax.set_ylim([30, 94])
        
axins = zoomed_inset_axes(ax, zoom= 3, bbox_to_anchor=(700, 500))
for i, row in r32.sort_values('k').iterrows():
    if not row.momentum:
        axins.plot(
            row.epoch_duration,
            row.accuracy
        )
        
ax.get_legend().set_title('K')
ax.get_legend()._set_loc(4)
        
axins.set_ylim([84, 91])
axins.set_xlim([1200, 2200])

# plt.xticks(visible=False)
# plt.yticks(visible=False)


mark_inset(ax, axins, loc1=2, loc2=1, ec="0.5");

plt.tight_layout()
plt.savefig('./figures/paper/resnet32/resnet32-32-comparison.pdf')

## Same with 128

In [ ]:
sns.set_palette('gnuplot2')

# get an intuition of the comparison with and without momentum by plotting the accuracy progression
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset

f, ax = plt.subplots(figsize=(14,10))

# plot all the lines
for _, row in r128.sort_values('k').iterrows():
    if not row.momentum:
        sns.lineplot(
            x=row.epoch_duration,
            y=row.accuracy,
            label=f'{int(row.k) if row.k != float("inf") else row.k}'
        )
        
# set the axes names
ax.set_xlabel('Time (sec)')
ax.set_ylabel('Accuracy (%)')
ax.set_ylim([30, 94])
        
axins = zoomed_inset_axes(ax, zoom= 2, bbox_to_anchor=(700, 500))
for i, row in r128.sort_values('k').iterrows():
    if not row.momentum:
        axins.plot(
            row.epoch_duration,
            row.accuracy
        )
        
ax.get_legend().set_title('K')
ax.get_legend()._set_loc(4)
        
axins.set_ylim([79, 86])
axins.set_xlim([700, 1100])

# plt.xticks(visible=False)
# plt.yticks(visible=False)


mark_inset(ax, axins, loc1=2, loc2=1, ec="0.5");

plt.tight_layout()
# plt.savefig('./figures/paper/resnet32/resnet32-32-comparison.pdf')

## Compare with the Momentum

In [ ]:
print(sns.color_palette('gnuplot2').as_hex())

In [ ]:
# get both of the entries with K = inf

sns.set_palette('gnuplot2')

inf = r32[r32.k==float('inf')]


f, ax = plt.subplots(figsize=(10,7))
# plot all the lines
for _, row in inf.sort_values('k').iterrows():

    color='#000090' if row.momentum else '#ff9669'
    label = 'SGD w/ Momentum' if row.momentum else 'SGD'
    
    sns.lineplot(
        x=row.epoch_duration,
        y=row.accuracy,
        label=label,
        color=color
    )
    
# set the axes names
ax.set_xlabel('Time [s]')
ax.set_ylabel('Accuracy [%]')
ax.set_ylim([30, 92])
    
        
# ax.get_legend().set_title('Momentum')
ax.get_legend()._set_loc(4)

plt.tight_layout()
plt.savefig('./figures/paper/resnet32/resnet32-momentum-comparison.pdf')

---

## Do ANOVA Linear Model to calculate the influence of the parameters

Using ANOVA we can get an idea of how the different parameters interact with each other and their influence

In [ ]:
# For the ANOVA test
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
def ANOVA(df: pd.DataFrame, y: str, use_all = False,verbose=False):
    """Run the ANOVA analysis with the batch, k and parallelism columns for the 
    given output variable"""
    
    # If use all is true we use all the variables to check either accuracy and time
    # including also the iowait and the cpu to see what fully influences the stuff
    
    
    if not use_all:
        # Plot the summary dataframe
        if verbose:
            display(rp.summary_cont(df.groupby(['batch_size', 'k', 'parallelism']))[y])

        model = ols(f'{y} ~ batch_size*k*parallelism', df).fit()
        
    else:
        if y not in ['acc', 'time']:
            raise ValueError('When use_all = True we predict either final_accuracy or time, not', y)
        if verbose:
            display(rp.summary_cont(df.groupby(['batch_size', 'k', 'parallelism']))[y])

        model = ols(f'{y} ~ cpu*batch*njobs*cpu_mean*iowait_mean', df).fit()
        

    if verbose:
        print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")
        display(model.summary())
    
    res = sm.stats.anova_lm(model, typ=2)
    
    return res, model

In [ ]:
# df.k = df.k.map(lambda val: -1 if val == float('inf') else val)

__r = r[r.system=='kubeml']
__r['k'] = __r['k'].map(lambda v: -1 if v == float('inf') else v)
res, model = ANOVA(__r, y='gpu_usage', verbose=True)

res

